In [ ]:
# from google.colab import drive
# drive.flush_and_unmount()           # ignore errors if already unmounted

#If cannot remount, simply delete the mounted drive and then remount
# rm -rf /content/drive


In [1]:
# Colab cell
from google.colab import drive

drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [2]:
# Adjust these two for YOUR repo
REPO_OWNER = "ywanglab"
REPO_NAME  = "STAT4160"   # e.g., unified-stocks-team1
BASE_DIR   = "/content/drive/MyDrive/dspt25"
CLONE_DIR  = f"{BASE_DIR}/{REPO_NAME}"
REPO_URL   = f"https://github.com/{REPO_OWNER}/{REPO_NAME}.git"

# if on my office computer

# REPO_NAME  = "lectureNotes"   # e.g., on my office computer
# BASE_DIR = r"E:\OneDrive - Auburn University Montgomery\teaching\AUM\STAT 4160 Productivity Tools" # on my office computer
# CLONE_DIR  = f"{BASE_DIR}\{REPO_NAME}"

import os, pathlib
pathlib.Path(BASE_DIR).mkdir(parents=True, exist_ok=True)


In [3]:
import os, subprocess, shutil, pathlib

if not pathlib.Path(CLONE_DIR).exists():
    !git clone {REPO_URL} {CLONE_DIR}
else:
    # If the folder exists, just ensure it's a git repo and pull latest
    os.chdir(CLONE_DIR)
    # !git status
    # !git pull --rebase # !git pull --ff-only
os.chdir(CLONE_DIR)
print("Working dir:", os.getcwd())

Working dir: /content/drive/MyDrive/dspt25/STAT4160


## Session 19 — Tensors, Datasets, Training Loop

### Learning goals

By the end of class, students can:

1.  Create a **windowed sequence dataset** across **multiple tickers** with shape `(B, T, F)` → predict `r_1d` at time `t+1`.
2.  Use `Dataset`/`DataLoader` correctly: **pin memory**, worker seeding, and efficient slicing.
3.  Write a **minimal training loop** with **early stopping**, **AMP** (mixed precision on GPU), and **checkpoint save/load**.
4.  Produce a tidy **validation metrics CSV** to compare later models.

------------------------------------------------------------------------

## Agenda

-   tensors & batching; `Dataset`/`DataLoader`; pinning memory; reproducibility with seeds

-   training loop anatomy; early stopping; AMP; checkpoints

-   **In‑class lab**: build `WindowedDataset` → DataLoaders → tiny GRU regressor → train w/ early stopping → save best checkpoint; evaluate

-   Wrap‑up + homework brief

------------------------------------------------------------------------

# ⭐Tensors & batching

-   Tensors are N‑D arrays on **CPU** or **GPU** (`.to(device)`); keep everything `float32` unless using AMP (Automatic mixed precision: Pytorch tool for faster training using FP16 where safe).

AMP is a PyTorch tool that:

* makes training **faster**
* uses **less GPU memory**
* and does this automatically by switching between FP16 and FP32 internally

All you do is wrap your forward pass with:

```python
with torch.autocast("cuda"):
    y = model(x)
    loss = criterion(y, target)
```

and use a **GradScaler** for safe gradients.

---



-   For sequences: **batch** × **time** × **features** ⇒ `(B, T, F)`. Predict a scalar per sequence end (`r_1d` at time `t+1`).

### `Dataset`/`DataLoader` patterns

-   **Precompute an index** of windows: for each ticker, sliding windows end at `i` with context `T`; target is `r_1d[i]`.

-   `DataLoader` tips:

    -   `pin_memory=True` and `non_blocking=True` on `.to(device)` speed H2D copies (when using GPU).

   

# ⭐In GPU computing:

* **Host (H)** = your **CPU** and system memory (RAM)
* **Device (D)** = your **GPU** and GPU memory (VRAM)

So whenever data needs to be used by the GPU, it must be **copied from the host to the device**.


**H2D** stands for:

**H → D = Host-to-Device**

So an **H2D copy** is:

> “Copying data from CPU memory into GPU memory.”

Example in PyTorch:

```python
x = x.to("cuda")     # this triggers an H2D copy
```

This takes time — often **more time than the actual GPU computation** — which is why  minimizing H2D copies.

---

Every time you do:

* `.to("cuda")`
* `tensor.cuda()`
* `tensor.to(device)`

you are performing an **H2D copy**.

If you do this inside your training loop for every batch, you slow things down a lot.

---

### ❓ If you have a DataLoader that outputs CPU tensors, and you call `.to("cuda")` on each batch inside your training loop, is that an H2D copy?

<details>
  <summary>Click to show answer</summary>

 **yes.**

</details>



And that leads to the core idea from Lecture 19:

> **One H2D copy per batch is fine. Many H2D copies per batch will kill your GPU speed.**


## ⭐ Mini-question

Suppose you accidentally write this inside your training loop:

```python
x = x.to("cuda")
y = y.to("cuda")
loss = loss.to("cuda")
model = model.to("cuda")
```

### ❓ How many of these should *actually* be on the GPU each iteration?

(Just pick one of these options)

**A.** All of them

**B.** Only `x` and `y`

**C.** Only the model

**D.** The model once (outside the loop) + `x` and `y` each batch

**E.** Everything should stay on CPU

Which one feels right?

<details>
  <summary>Click to show answer</summary>

Correct answer: **D**
###  The **model** should be moved to GPU **once**, before the training loop:

```python
model = model.to("cuda")
```

(If you do this inside the loop, you pay the H2D copy cost **every batch**, which is extremely slow.)

  Each batch’s **x** and **y** need to be copied H→D:

```python
x = x.to("cuda")
y = y.to("cuda")
```

 The **loss** should *not* be moved; it will already live on the GPU because `y_pred` and `y` are on GPU.

---

 Why this matters (1-sentence intuition)

> **Good GPU training = copy as little as possible.**
> You want GPU data to *stay* on the GPU.


</details>









    -   Seed workers for reproducibility; keep `num_workers=2` (Colab stable).

### ⭐Seeds & determinism

-   Set `python`, `numpy`, `torch` seeds; disable CuDNN benchmarking for reproducibility; prefer small batch sizes that fit CPU/GPU.

### ⭐Training loop w/ early stopping

-   Loop: `train_step` (model in `train()`), `val_step` (model in `eval()` + `no_grad()`).
-   Track **best val loss**; stop after `patience` epochs without improvement.
-   Save **checkpoint**: `state_dict`, optimizer state, epoch, metrics. Load with `load_state_dict`.

### ⭐AMP & checkpoints

-   On CUDA, wrap forward in `torch.cuda.amp.autocast()` and use `GradScaler` to scale loss.
-   Save the best checkpoint to `models/…pt`; log a CSV under `reports/`.

------------------------------------------------------------------------

## In‑class lab (Colab‑friendly)

> Run each block as its own **separate cell** in Colab. Replace `REPO_NAME` as needed.

### 0) Setup, mount, and check device


In [8]:
import os, pathlib, sys

for p in ["data/processed","models","reports","scripts","tests"]:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)
print("Working dir:", os.getcwd())

import torch, platform
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available(), "| Python:", sys.version.split()[0], "| OS:", platform.system())

Working dir: /content/drive/MyDrive/dspt25/STAT4160
Torch: 2.8.0+cu126 | CUDA available: False | Python: 3.12.12 | OS: Linux


⭐
```python
sys.version.split()[0]
```

`sys.version`

This is a **string** describing your Python version, e.g.:

```
'3.10.12 (main, Jul  6 2023, 12:00:00) [GCC 11.2.0]'
```

`.split()`

Splitting that string (default = split on spaces) gives a list:

```
['3.10.12',  
 '(main,',  
 'Jul',  
 '6',  
 '2023,',  
 '12:00:00)',  
 '[GCC',  
 '11.2.0]']
```

 `[0]`

This takes the **first element**:

```
'3.10.12'
```

So the entire expression:

```python
sys.version.split()[0]
```

returns **just the Python version number**, without any extra metadata.


### ❓ If `sys.version` were the string

```
"3.9.6 something something"
```

what would `sys.version.split()[1]` return?

<details>
  <summary>Click to show answer</summary>

"something"

</details>



# ⭐ What `platform.system()` does

It returns a **string** describing the **operating system** your Python interpreter is running on.

Typical outputs:

* `"Windows"`
* `"Linux"`
* `"Darwin"` (this is macOS)



# ⭐ Mini-check (your turn)

If you run `platform.system()` inside Google Colab, what do you expect it prints?

Just pick one:

A. `"Windows"`
B. `"Linux"`
C. `"Darwin"`
<details>
  <summary>Click to show answer</summary>

B

</details>


# 1) Load features and pick columns (with fallbacks)

In [9]:
import pandas as pd, numpy as np
from pathlib import Path

# Prefer static universe file if present (from Session 17)
f_static = Path("data/processed/features_v1_static.parquet")
f_base   = Path("data/processed/features_v1.parquet")

if f_static.exists():
    df = pd.read_parquet(f_static)
elif f_base.exists():
    df = pd.read_parquet(f_base)
else:
    # Minimal fallback from returns
    rpath = Path("data/processed/returns.parquet")
    if not rpath.exists():
        # synthesize small dataset
        rng = np.random.default_rng(0)
        dates = pd.bdate_range("2022-01-03", periods=320)
        frames=[]
        for t in ["AAPL","MSFT","GOOGL","AMZN","NVDA"]:
            eps = rng.normal(0, 0.012, size=len(dates)).astype("float32")
            adj = 100*np.exp(np.cumsum(eps))
            g = pd.DataFrame({
                "date": dates, "ticker": t,
                "adj_close": adj.astype("float32"),
                "log_return": np.r_[np.nan, np.diff(np.log(adj))].astype("float32")
            })
            g["r_1d"] = g["log_return"].shift(-1)
            frames.append(g)
        df = pd.concat(frames, ignore_index=True).dropna().reset_index(drop=True)
        df["ticker"] = df["ticker"].astype("category")
    else:
        df = pd.read_parquet(rpath)
        df = df.sort_values(["ticker","date"]).reset_index(drop=True)
        # add minimal lags
        for k in [1,2,3]:
            df[f"lag{k}"] = df.groupby("ticker")["log_return"].shift(k)
        df = df.dropna().reset_index(drop=True)

# Ensure minimal features exist
cand_feats = ["log_return","lag1","lag2","lag3","zscore_20","roll_std_20"]
FEATS = [c for c in cand_feats if c in df.columns]
assert "r_1d" in df.columns, "Label r_1d missing; rebuild returns/features pipeline."
assert "log_return" in df.columns, "log_return missing."

# Keep a small subset of tickers for speed (5–10 tickers)
subset = df["ticker"].astype(str).unique().tolist()[:8]
df = df[df["ticker"].astype(str).isin(subset)].copy()

# Harmonize types & sort
df["date"] = pd.to_datetime(df["date"])
df["ticker"] = df["ticker"].astype("category")
df = df.sort_values(["ticker","date"]).reset_index(drop=True)

print("Using features:", FEATS, "| tickers:", df["ticker"].nunique(), "| rows:", len(df))
df.head()

Using features: ['log_return', 'lag1', 'lag2', 'lag3', 'zscore_20', 'roll_std_20'] | tickers: 8 | rows: 1272


,date,ticker,log_return,r_1d,weekday,month,lag1,lag2,lag3,roll_mean_20,roll_std_20,zscore_20,ewm_mean_20,ewm_std_20,exp_mean,exp_std,adj_close,volume
0,2020-01-29,AAPL,-0.018417,-0.002351,2,1,-0.012895,-0.019012,-0.004576,-0.004086,0.008476,-1.690830,-0.005252,0.009304,-0.004086,0.008476,92.154846,1598707
1,2020-01-30,AAPL,-0.002351,-0.012675,3,1,-0.018417,-0.012895,-0.019012,-0.004353,0.008324,0.240455,-0.004976,0.008875,-0.004003,0.008270,91.938454,2992900
2,2020-01-31,AAPL,-0.012675,0.002713,4,1,-0.002351,-0.018417,-0.012895,-0.004849,0.008517,-0.918756,-0.005709,0.008745,-0.004397,0.008280,90.780533,634335
3,2020-02-03,AAPL,0.002713,0.001568,0,2,-0.012675,-0.002351,-0.018417,-0.004268,0.008622,0.809695,-0.004907,0.008690,-0.004088,0.008224,91.027122,913454
4,2020-02-04,AAPL,0.001568,-0.001869,1,2,0.002713,-0.012675,-0.002351,-0.003963,0.008719,0.634254,-0.004291,0.008486,-0.003852,0.008126,91.169922,662663


# Time‑based split (first rolling‑origin split with embargo)

In [10]:
def make_rolling_origin_splits(dates, train_min=252, val_size=63, step=63, embargo=5):
    u = np.array(sorted(pd.to_datetime(pd.Series(dates).unique())))
    i = train_min - 1; out=[]
    while True:
        if i >= len(u): break
        a,b = u[0], u[i]
        vs = i + embargo + 1
        ve = vs + val_size - 1
        if ve >= len(u): break
        out.append((a,b,u[vs],u[ve]))
        i += step
    return out

splits = make_rolling_origin_splits(df["date"], 80, 21, 21, 5)
assert splits, "Not enough history for a first split."
a,b,c,d = splits[0]
train_df = df[(df["date"]>=a)&(df["date"]<=b)].copy()
val_df   = df[(df["date"]>=c)&(df["date"]<=d)].copy()
print("Split 1 - Train:", a.date(), "→", b.date(), "| Val:", c.date(), "→", d.date(),
      "| train rows:", len(train_df), "| val rows:", len(val_df))

Split 1 - Train: 2020-01-29 → 2020-05-19 | Val: 2020-05-27 → 2020-06-24 | train rows: 640 | val rows: 168


### 3) Reproducibility helpers, `FeatureScaler`, and `WindowedDataset`

In [12]:
import random, math, json

def seed_everything(seed=1337):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
seed_everything(1337)

class FeatureScaler: #customized version similar to StandardScaler of scikit-learn
    """Train-only mean/std scaler for numpy arrays.""" # fitting on the val data woould leak information
    def __init__(self): self.mean_=None; self.std_=None
    def fit(self, X: np.ndarray):                    #addtributes endign in _ are learned from data, same convention used in scikit-learn.
        self.mean_ = X.mean(axis=0, dtype=np.float64) #X.shape=(T,F), mean(axis=0): feature-wise means->shape(F,)
        self.std_  = X.std(axis=0, dtype=np.float64) + 1e-8
        return self
    def transform(self, X: np.ndarray) -> np.ndarray:
        return (X - self.mean_) / self.std_
    def state_dict(self):
        return {"mean": self.mean_.tolist(), "std": self.std_.tolist()} #np array-> Python lists->JSON friendly
    def load_state_dict(self, d):   #restore the stats from d (for example, read from disk)
        self.mean_ = np.array(d["mean"], dtype=np.float64)
        self.std_  = np.array(d["std"],  dtype=np.float64)

from torch.utils.data import Dataset, DataLoader

class WindowedDataset(Dataset):
    """
    Sliding windows over time per ticker (multi-ticker, fixed context_len).
    Each item: X in shape (T, F), y scalar: r_1d at window end.
    """
    def __init__(self, frame: pd.DataFrame, feature_cols, context_len=64, scaler: FeatureScaler|None=None):
        assert "ticker" in frame and "date" in frame and "r_1d" in frame
        self.feature_cols = feature_cols
        self.T = int(context_len)
        self.groups = {}   # ticker -> dict('X': np.ndarray [N,F], 'y': np.ndarray [N])
        self.index  = []   # list of (ticker, end_idx)
        # Build groups (per ticker)
        for tkr, g in frame.groupby("ticker"):
            g = g.sort_values("date").reset_index(drop=True)
            X = g[feature_cols].to_numpy(dtype=np.float32)
            y = g["r_1d"].to_numpy(dtype=np.float32)
            # valid windows end where we have T steps and y is finite
            for end in range(self.T-1, len(g)):
                if not np.isfinite(y[end]): # isfinite(value) returns true if value is not NaN, +inf, -inf. ONly windows whose target is a valid finite float are included.
                    continue
                self.index.append((tkr, end))
            self.groups[tkr] = {"X": X, "y": y}
        # Fit scaler on all TRAIN rows (only when building train dataset)
        self.scaler = scaler or FeatureScaler().fit( # use scaler or creae and fit a new FeatureScaler on this frame
            np.concatenate([self.groups[t]["X"] for t in self.groups], axis=0)
        )
    def __len__(self): return len(self.index)
    def __getitem__(self, i):
        tkr, end = self.index[i]
        g = self.groups[tkr]
        xs = g["X"][end-self.T+1:end+1]        # (T, F) context
        xs = self.scaler.transform(xs)         # scale using train stats
        y  = g["y"][end]                       # scalar target
        return torch.from_numpy(xs), torch.tensor(y), str(tkr)

def make_loaders(train_df, val_df, feature_cols, context_len=64, batch_size=256, num_workers=2):
    # Train dataset fits scaler; Val shares it
    train_ds = WindowedDataset(train_df, feature_cols, context_len=context_len, scaler=None) # scaler=None: the data set fits a new scaler on the trianing data
    val_ds   = WindowedDataset(val_df,   feature_cols, context_len=context_len, scaler=train_ds.scaler)
    # Persist scaler for reuse
    Path("models").mkdir(exist_ok=True)
    Path("models/scaler_split1.json").write_text(json.dumps(train_ds.scaler.state_dict())) # save the scaler for future inference.
    pin = torch.cuda.is_available()
    g = torch.Generator()
    g.manual_seed(42)
    def _seed_worker(_):
        worker_seed = torch.initial_seed() % (2**32)
        np.random.seed(worker_seed); random.seed(worker_seed)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True,
                              num_workers=num_workers, pin_memory=pin, persistent_workers=(num_workers>0),
                              worker_init_fn=_seed_worker, generator=g)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False,
                              num_workers=num_workers, pin_memory=pin, persistent_workers=(num_workers>0),
                              worker_init_fn=_seed_worker)
    return train_ds, val_ds, train_loader, val_loader

train_ds, val_ds, train_loader, val_loader = make_loaders(train_df, val_df, FEATS, context_len=32, batch_size=32)

len(train_ds), len(val_ds), len(train_loader), len(val_loader), next(iter(train_loader))[0].shape

/tmp/ipython-input-142154883.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for tkr, g in frame.groupby("ticker"):
/tmp/ipython-input-142154883.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for tkr, g in frame.groupby("ticker"):


(392, 0, 12, 0, torch.Size([32, 32, 6]))

# ⭐ What `seed_everything()` does (big picture)

It ensures **reproducibility**.

When you train a neural network, many parts involve randomness:

* shuffling data
* initializing weights
* dropout masks
* GPU kernels picking different execution paths
* multi-worker DataLoader randomness

If you **seed** everything, you make each run produce the **same results** every time — same batches, same weights, same loss curve.

---

# ⭐ Line-by-line explanation

## 1. `random.seed(seed)`

This seeds Python’s built-in random number generator.

Affects things like:

```python
random.random()
random.shuffle()
random.randint()
```

So your **shuffling** or **random splits** won’t change each run.

---

## 2. `np.random.seed(seed)`

Seeds NumPy's random generator.

Affects:

* random number generation in NumPy
* bootstraps
* sampling
* any NumPy-based augmentation

This ensures all NumPy randomness is reproducible.

---

## 3. `torch.manual_seed(seed)`

Seeds PyTorch’s random generator **on CPU**.

Controls:

* weight initialization
* dropout masks
* any CPU tensor randomness

---

## 4. `torch.cuda.manual_seed_all(seed)`

If a GPU is available, this seeds **ALL GPUs** (in case of multi-GPU training).

Controls:

* GPU tensor randomness
* CUDA RNG kernels
* dropout on GPU

---

## 5. `torch.backends.cudnn.benchmark = False`

cuDNN wants to “auto-tune” convolution algorithms for faster speed.

This auto-tuning introduces **non-determinism**.

Setting `benchmark=False` disables that tuning → more consistent behavior.

---

## 6. `torch.backends.cudnn.deterministic = True`

Forces cuDNN to use **deterministic algorithms** only.

Some GPU ops have fast algorithms that are nondeterministic.
This makes them deterministic — but sometimes a bit slower.


---

# ⭐ Quick check

❓ If you **don’t** call `seed_everything()`, which of these will vary between runs?

A. Weight initialization

B. Dropout masks

C. Shuffle order of DataLoader batches

D. GPU random operations

E. All of the above

<details>
  <summary>Click to show answer</summary>

E

</details>


## ⭐ Tiny reinforcement check

Just to solidify your understanding:

### ❓ Why does setting

```python
torch.backends.cudnn.deterministic = True
```

make training slightly slower?

Pick one:

A. It forces cuDNN to use deterministic algorithms that might not be the fastest.
B. It keeps the GPU in CPU mode.
C. It disables the optimizer.
D. It runs all operations twice.

Which one feels right?


<details>
  <summary>Click to show answer</summary>

 **A**

Deterministic ≠ fastest.
cuDNN often has multiple algorithms for convolution, pooling, etc.:

* Some are fast but nondeterministic
* Some are slower but deterministic

When you set:

```python
torch.backends.cudnn.deterministic = True
```

PyTorch forces cuDNN to pick the **deterministic-but-slower** ones.

That’s the tradeoff:

> **Perfect reproducibility → slight speed loss.**


</details>


# ⭐ What `FeatureScaler` does (big picture)

It is a **very small custom version** of scikit-learn’s `StandardScaler`:

* It computes **mean** and **std** from the **training** set.
* It stores them.
* It applies the same normalization to **train** and **val** windows.
* It can save/load the scaling stats (`state_dict`).

We do NOT fit the scaler on the **validation** data — that would leak information.


Because the model uses a **sliding window** per ticker:

* If we normalize per window → we leak future information
* If we normalize per ticker → different tickers have mismatched scales
* If we normalize per split → val is contaminated by train

Thus:

> **We compute mean/std on ALL training rows across ALL tickers — only once.**

Then reuse these stats for val/test/inference.



# ⭐ Small check

Look at this line:

```python
self.std_ = X.std(axis=0, dtype=np.float64) + 1e-8
```

### ❓ Why do you think we add `1e-8` to the std?

A. To make training faster

B. To avoid dividing by zero

C. To improve GPU performance

D. To increase randomness


Which one feels right?
<details>
  <summary>Click to show answer</summary>

 B
</details>


⭐ ## Tiny check

❓ Why do we **reuse** the same `scaler` for the validation dataset instead of fitting a new one on `val_df`?

Try to say it in your own words (even a short sentence is fine).
<details>
  <summary>Click to show answer</summary>

We must **not leak information from the validation set** into the training process.

If you fit the scaler on validation data:

* the mean/std of validation features influence the model
* so the model “peeks” at future data
* this destroys the purpose of validation
* and gives overly optimistic evaluation results

So the rule is:

> **Fit normalization only on training data.
> Apply (reuse) that normalization on val/test/inference data.**

</details>


# ⭐ Quick reinforcement (small question)

Look at this line again:

```python
np.concatenate([self.groups[t]["X"] for t in self.groups], axis=0)
```

### ❓ Why do we concatenate all tickers’ `X` values before fitting the scaler,

instead of fitting a separate scaler **per ticker**?

Choose the best reason:

A. Separate scalers per ticker would leak information.

B. Tick-by-tick scaling would make features inconsistent across tickers.

C. Concatenation makes training faster.

D. PyTorch requires it.


Which one do you think is correct?

<details>
  <summary> click to see the answer. </summary>
   **B **

If we used **one scaler per ticker**, then:

* AAPL features would be normalized using AAPL’s own mean/std
* MSFT features using MSFT’s mean/std
* NVDA using NVDA’s mean/std
* etc.

That means:

> The same model input feature (e.g., volume, return) would be on **different scales** depending on which ticker it came from.

The neural network would then struggle because:

* A “0.1” feature from AAPL means something **different** from a “0.1” feature from MSFT.
* Feature ranges vary ticker-to-ticker.
* The model must learn two things at once:

  * the pattern
  * the rescaling needed for each ticker

This makes training harder and slower.

So we instead compute **one global mean** and **one global std** across all tickers.

This ensures:

* All tickers sit in the **same feature space**
* Data is comparable across tickers
* The model doesn't accidentally learn scaling instead of patterns

</details>



⭐❓ Suppose MSFT has 10× larger dollar volume than AAPL.
After global scaling, will the model still “see” that difference?

A. Yes

B. No

C. Only during training

D. Only if we add a special feature

What do you think?

<details>
<summary> click to see the answer </summary>

 **A — Yes**, the model still sees the difference.



### ✔ What scaling *removes*

Global standardization removes:

* absolute magnitude
* raw units
* arbitrary differences like “MSFT volume is expressed in millions, AAPL in thousands”

### ✔ What scaling *keeps*

Scaling **does not remove the pattern** that MSFT tends to have higher volume than AAPL.

Why?

Because standardization (z-scoring):

[
x' = \frac{x - \mu}{\sigma}
]

preserves **relative differences**:

* If MSFT volume is frequently above the global mean → its z-scores stay **positive**.
* If AAPL volume is usually lower → its z-scores stay **negative or smaller**.

So the *ranking* and *relative position* are preserved, even though the raw numbers are not.

---

#  Quick intuition example

Suppose global mean volume = 5M shares.

| Ticker | Actual Volume | After Scaling |
| ------ | ------------- | ------------- |
| MSFT   | 12M           | high positive |
| AAPL   | 3M            | negative      |

Even after scaling:

* MSFT still looks “bigger”
* AAPL still looks “smaller”

The neural network sees this difference clearly.


</details>
# ⭐ Mini-check (one quick question)

If two features are perfectly correlated before scaling (one is exactly 2× the other),
after scaling they will be:

A. still perfectly correlated\
B. only partly correlated\
C. uncorrelated\
D. random relative to each other

What do you think?
<details>
<summary> Click for answer </summary>
 **A is correct.**



If two features satisfy:

$$
x_2 = 2x_1
$$

then when you standardize each:

$$
x_1' = \frac{x_1 - \mu_1}{\sigma_1},\quad
x_2' = \frac{x_2 - \mu_2}{\sigma_2}
$$

the relationship becomes:

$$
x_2' = \frac{2x_1 - \mu_2}{\sigma_2}
$$

But since:

* $\mu_2 = 2\mu_1$
* $\sigma_2 = 2\sigma_1$

we get:

$$
x_2' = \frac{2(x_1 - \mu_1)}{2\sigma_1}
= \frac{x_1 - \mu_1}{\sigma_1}
= x_1'
$$

So after scaling:

$$
x_2' = x_1'
$$


</details>

## ⭐ Mini-Check To Lock It In

Suppose we have:

* AAPL’s 10-day average volume tends to be slightly below the global mean.
* MSFT’s tends to be above the global mean.

After global scaling…

### ❓ Which will happen?

A. AAPL’s normalized volume tends to be negative; MSFT’s tends to be positive
B. The model cannot tell them apart
C. All tickers will have zero-mean volume on every individual day
D. Scaling destroys all volume information

Which one feels right **now that you understand the mechanism**?
<details>
<summary> click for answer </summary>
**A**
</detials>






# ⭐ Build PyTorch datasets and Dataloaders

# 1. Build the two datasets

```python
train_ds = WindowedDataset(..., scaler=None)
val_ds   = WindowedDataset(..., scaler=train_ds.scaler)
```

###  Train dataset

*  `scaler=None`, which means the dataset **fits a new scaler** on the training features.
* This scaler is saved as `train_ds.scaler`.

###  Validation dataset

*  `scaler=train_ds.scaler`
* So validation windows use **the same mean/std** as the train set.
* This avoids data leakage

---

# ⭐ 2. Save the scaler to disk

```python
Path("models").mkdir(exist_ok=True)
Path("models/scaler_split1.json").write_text(json.dumps(train_ds.scaler.state_dict()))
```

Because later, when you restore your model for inference on new data,
you must use **the same scaling statistics**.

This saves the scaler as a JSON file like:

```json
{"mean": [...], "std": [...]}
```


# ⭐ 3. Setup for DataLoader

### `pin_memory = torch.cuda.is_available()`

If CUDA is available:

* DataLoader uses **page-locked host memory**,
* which makes **H→D copies faster**.


---

# ⭐ 4. Create a manual random generator

```python
g = torch.Generator()
g.manual_seed(42)
```

This ensures **DataLoader shuffling is reproducible** across runs.

Without this, even if you `seed_everything()`,
**DataLoader workers** may shuffle batches differently.

---

# ⭐ 5. Worker seeding

```python
def _seed_worker(_):
    worker_seed = torch.initial_seed() % (2**32)
    np.random.seed(worker_seed)
    random.seed(worker_seed)
```

When using `num_workers > 0`, each worker process must be seeded,
otherwise:

* batch shuffling becomes nondeterministic

* dropout masks produced on CPU become inconsistent

This function ensures **each worker gets its own deterministic seed**.

---

# ⭐ 6. Build train_loader

```python
train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
    pin_memory=pin,
    persistent_workers=(num_workers>0),
    worker_init_fn=_seed_worker,
    generator=g
)
```

Key points:

###  `shuffle=True`

Randomizes order of batches every epoch.

###  `drop_last=True`

Drops the last partial batch.

* `len(train_ds) < batch_size` → **no batches at all**
* this is why you saw `StopIteration`

###  `persistent_workers=True`

Keeps workers alive between epochs → faster reload.

###  `pin_memory`

Speeds up CPU→GPU transfers.

###  `worker_init_fn` + `generator`

Makes shuffling + worker randomness reproducible.

---

# ⭐ 7. Build val_loader

```python
val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=pin,
    persistent_workers=(num_workers>0),
    worker_init_fn=_seed_worker
)
```

Differences from train_loader:

* No shuffling (`shuffle=False`)
* No dropping incomplete batch (`drop_last=False`)

Validation must be deterministic and evaluate **all** data.

---



# ⭐ Mini-check

### ❓ Why do we use

```python
shuffle=True
```

for the **training** DataLoader but

```python
shuffle=False
```

for the **validation** DataLoader?

<details>
<summary> click for answer </summary>

Why `shuffle=True` for training

During training, we want batches to be:

* independent
* randomly mixed
* not in chronological/ticker order
* not grouped by similar values

Why?

Because shuffling:

* prevents the model from overfitting to ordering patterns
* improves gradient estimates
* reduces variance
* helps SGD converge smoothly
* avoids “bad luck” batches (e.g., all rising stocks in one batch)


> **Training = shuffle to help the model generalize.**



 Why `shuffle=False` for validation

Validation must be:

* deterministic
* repeatable
* stable from epoch to epoch
* in the same order so metrics can be compared

If validation were shuffled:

* the order changes, so evaluation would vary
* metrics like MAE might change slightly run-to-run
* debugging becomes harder
* reproducibility is lost


> **Validation = NO shuffle, because we want a consistent measurement.**

---

 summary in one sentence

> **Shuffle training to improve learning; keep validation ordered to measure performance consistently.**


</details>


# ⭐ 1. What is a **worker** (in DataLoader)?

**A worker = a separate background process created by DataLoader**
when you set:

```python
num_workers > 0
```

Each worker:

* loads data
* runs your `Dataset.__getitem__`
* prepares batches
* feeds batches to the main process

This makes loading **much faster**, because workers run **in parallel**.

### Example

If you set:

```python
DataLoader(..., num_workers=4)
```

Then the DataLoader creates **4 separate processes**.
These 4 workers load batches at the same time.

### Why we need worker a seed?

Because each worker has its own random state.
Without seeding, each worker would shuffle, augment, or sample **differently every run**.



# ⭐ 2. What is `torch.Generator`?

A **torch.Generator** is an object that *stores a random-number generator state*.
You can think of it as “a private RNG for PyTorch.”

Example:

```python
g = torch.Generator()
g.manual_seed(42)
```

This creates a **separate, independent RNG** that DataLoader can use for shuffling.

If we did not use our own generator:

* DataLoader shuffling can vary between runs
* Even if you call `torch.manual_seed`, DataLoader spawns new worker processes → new seeds → different order

So this is the **correct and safe** way to force DataLoader to shuffle deterministically.

---

# ⭐ 3. Difference between

### `torch.Generator().manual_seed(seed)`

### `torch.initial_seed()`

### `torch.manual_seed(seed)`


---

##  **A. `torch.manual_seed(seed)`**

Seeds **the global PyTorch RNG on the main process**.

Affects:

* weight initialization
* dropout masks
* CPU ops with randomness
* GPU ops with randomness (only for the current device)

You usually call:

```python
torch.manual_seed(1337)
```

But DataLoader workers do **not** inherit this seed predictably — they fork from the parent process and get modified seeds.

---

##  **B. `torch.Generator().manual_seed(seed)`**

Creates a **separate RNG object**, with its own seed.

Example:

```python
g = torch.Generator()
g.manual_seed(42)
```

This is mainly used when you want:

* reproducible DataLoader shuffling
* reproducible random sampling inside a function
* an RNG that is separate from the global one

When you pass it into DataLoader:

```python
DataLoader(..., generator=g)
```

you get **deterministic shuffling**.



##  **C. `torch.initial_seed()`**

This is called **inside a worker process**.

It returns:

> the seed assigned to this worker by PyTorch

This is why worker seeding functions use:

```python
worker_seed = torch.initial_seed() % (2**32)
```

Each worker gets a *different*, *deterministic* seed, created from the global seed and worker ID.

This seed is used to seed:

* NumPy
* Python’s `random`

inside that worker.

---

# ⭐ Summary Table (super simple)

| Function                           | Where it applies | What it seeds                          |
| ---------------------------------- | ---------------- | -------------------------------------- |
| `torch.manual_seed(s)`             | main process     | global PyTorch RNG                     |
| `torch.Generator().manual_seed(s)` | private RNG      | DataLoader shuffle or custom RNG state |
| `torch.initial_seed()`             | worker process   | returns worker’s assigned seed         |

---

# ⭐ One-sentence summary

> **`torch.manual_seed` seeds PyTorch globally,
> `torch.Generator().manual_seed` seeds a private RNG (often for DataLoader),
> and `torch.initial_seed()` tells each worker its assigned seed.**

---

# ⭐ Quick check (your turn)

### ❓ Why do we need both

`generator=g` **and** `worker_init_fn=_seed_worker`
in the DataLoader?

Pick the best explanation:

A. Generator controls global randomness; worker_init_fn adds speed.\
B. Generator seeds shuffling; worker_init_fn seeds each worker process.\
C. One is for CPU, the other is for GPU.\
D. Both do the same thing; one is redundant.\

What do you think?
<details>
<summary> Click for answer </summary>

 **B is correct.**


---


### ✔ `generator=g`

Controls **the shuffling order** of the DataLoader.

* Ensures batch order is identical each run
* Applies before worker processes are created
* Guarantees reproducible sampling

Think of it as:

> “What order should batches come in?”

---

###  `worker_init_fn=_seed_worker`

Seeds **each worker process** (Python + NumPy RNG).

* Ensures each worker samples deterministically
* Makes random transforms / sampling within the worker reproducible
* Prevents workers from generating different data each run

Think of it as:

> “Inside each worker, how should random numbers behave?”

---

#  Together, they guarantee FULL reproducibility

You need **both** pieces:

1. **`generator`** → controls *global* behavior (shuffling)
2. **`worker_init_fn`** → controls *per-worker* behavior (random ops in workers)

Without `generator`:
You cannot guarantee the same shuffle order.

Without `worker_init_fn`:
Each worker will produce different per-item randomness every run.

With both:
Your entire data pipeline becomes **completely deterministic**.



</details>


 # ⭐ why both `val_ds` and `val_loader` have length zero?


In [13]:
len(train_df), len(val_df)

(640, 168)

In [16]:
val_df.groupby("ticker").size()

/tmp/ipython-input-691067041.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_df.groupby("ticker").size()


,0
ticker,
AAPL,21
AMZN,21
BAC,21
CSCO,21
CVX,21
DIS,21
GOOGL,21
HD,21
INTC,0


In [14]:
val_df.groupby("ticker").size().describe()

/tmp/ipython-input-3518275911.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_df.groupby("ticker").size().describe()


,0
count,25.000
mean,6.720
std,9.998
min,0.000
25%,0.000
50%,0.000
75%,21.000
max,21.000


In [18]:
# Fix
# A ticker contributes a window (T) only when 1. there are at least T=context_len rows in g; 2. And y[end]=r_1d is finite.
# Adjust the context size.
train_ds, val_ds, train_loader, val_loader = make_loaders(train_df, val_df, FEATS, context_len=16, batch_size=32)

len(train_ds), len(val_ds), len(train_loader), len(val_loader), next(iter(train_loader))[0].shape

/tmp/ipython-input-142154883.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for tkr, g in frame.groupby("ticker"):
/tmp/ipython-input-142154883.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for tkr, g in frame.groupby("ticker"):


(520, 48, 16, 2, torch.Size([32, 16, 6]))

# ⭐ 4) Define a tiny GRU regressor

A GRU (Gated Recurrent Unit) model is a type of recurrent neural network (RNN) designed for processing sequential data, like text or time series. It addresses the limitations of standard RNNs by using a gating mechanism to manage the flow of information, making it more effective at capturing long-term dependencies. Compared to LSTM, the GRU is a simpler and more computationally efficient architecture, with fewer parameters, making it faster to train.  [1, 2, 3, 4, 5, 6]  
You can watch this video to learn about the working of GRU in detail: https://www.youtube.com/watch?v=IBs8D8PWMc8 (https://www.youtube.com/watch?v=IBs8D8PWMc8)
Key features of a GRU model:

• Gating mechanism: GRUs use two gates—an update gate and a reset gate—to control what information is remembered, forgotten, or passed on. [2, 5, 6]  
	• Update gate: Determines how much of the previous hidden state to keep and how much of the new candidate state to add, balancing past and present information. [2, 7]  
	• Reset gate: Decides how much of the previous information to forget, allowing the network to disregard irrelevant past information. [2, 6, 8]  

• Simplified architecture: It merges the forget and input gates into a single update gate and does not have a separate cell state like LSTMs, resulting in fewer parameters. [2, 4, 5]  
• Efficiency: Due to its simpler structure, the GRU model is often faster to train than an LSTM while performing similarly on many tasks. [4, 5, 9]  
• Sequential data processing: It excels at handling sequential data by maintaining a hidden state that summarizes information from previous steps. [3, 10]  

Common applications:

• Natural Language Processing (NLP): Used for tasks like text classification, sentiment analysis, and machine translation. [3, 11, 12]  
• Time series forecasting: Effective for predicting future values in a sequence, such as stock prices or weather patterns. [3, 13]  
• Speech recognition: Models the temporal dependencies in speech signals to improve accuracy. [1, 3]  


[1] https://en.wikipedia.org/wiki/Gated_recurrent_unit
[2] https://medium.com/@maizi5469/5-0-gated-recurrent-unit-gru-4bcd3065734f
[3] https://ravjot03.medium.com/gru-explained-the-simplified-rnn-solution-for-sequential-data-c706d0d149c5
[4] https://medium.com/@anishnama20/understanding-gated-recurrent-unit-gru-in-deep-learning-2e54923f3e2
[5] https://medium.com/@deepanshusnpt/day-10-gated-recurrent-unit-gru-networks-a-simplified-rnn-56aedfa3faf4
[6] https://www.analyticsvidhya.com/blog/2021/03/introduction-to-gated-recurrent-unit-gru/
[7] https://www.activeloop.ai/resources/glossary/gated-recurrent-units-gru/
[8] https://towardsdatascience.com/the-math-behind-gated-recurrent-units-854d88aded65/
[9] https://stackoverflow.com/questions/59932978/which-one-is-faster-either-gru-or-lstm
[10] https://www.mdpi.com/2073-4441/17/21/3039
[11] https://schneppat.com/gated-recurrent-unit-gru.html
[12] https://www.pickl.ai/blog/gated-recurrent-unit-in-deep-learning/
[13] https://medium.com/@chandramouliarun/gated-recurrent-units-grus-a-deep-dive-into-modern-sequence-modeling-4a90086101e9




In [20]:
import torch.nn as nn, torch

class GRURegressor(nn.Module):
    def __init__(self, in_features: int, hidden: int = 64, num_layers: int = 2, dropout: float = 0.1):
        super().__init__()
        self.gru = nn.GRU(input_size=in_features, hidden_size=hidden, num_layers=num_layers,
                          batch_first=True, dropout=dropout if num_layers>1 else 0.0)
        self.head = nn.Sequential(
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1)
        )
    def forward(self, x):        # x: (B, T, F)
        _, hN = self.gru(x)      # hN: (num_layers, B, H); take last layer
        h = hN[-1]               # (B, H)
        return self.head(h).squeeze(-1)  # (B,)

def make_model():
    return GRURegressor(in_features=len(FEATS), hidden=64, num_layers=2, dropout=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = make_model().to(device)
sum(p.numel() for p in model.parameters()), device

(43009, device(type='cpu'))

## ⭐ 1. The GRURegressor class

```python
class GRURegressor(nn.Module):
    def __init__(self, in_features: int, hidden: int = 64, num_layers: int = 2, dropout: float = 0.1):
        super().__init__()
```

* Subclass of `nn.Module` → standard PyTorch model.
* `in_features` = number of input features per time step (this will be `len(FEATS)`).
* `hidden` = hidden size of the GRU.
* `num_layers` = how many GRU layers stacked.
* `dropout` = regularization strength.

---

### 1a. The GRU layer

```python
self.gru = nn.GRU(
    input_size=in_features,
    hidden_size=hidden,
    num_layers=num_layers,
    batch_first=True,
    dropout=dropout if num_layers>1 else 0.0
)
```

* `input_size=in_features` → each time step has that many features.
* `hidden_size=hidden` → GRU hidden dimension = H.
* `num_layers=2` → stacked GRU with 2 layers.
* `batch_first=True` → input shape is `(B, T, F)` instead of `(T, B, F)`.
* Dropout between GRU layers only applies if `num_layers > 1` (PyTorch requirement).

So this GRU consumes a sequence and gives you:

* `output`: `(B, T, H)`
* `hN`: `(num_layers, B, H)` → final hidden state for each layer.

---

### 1b. The head (regression on the final state)

```python
self.head = nn.Sequential(
    nn.Linear(hidden, hidden),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(hidden, 1)
)
```

This is a small MLP that maps from the final GRU hidden state (size H) to a scalar prediction:

* `Linear(hidden, hidden)` → dense layer.
* `ReLU()` → nonlinearity.
* `Dropout` → regularization.
* `Linear(hidden, 1)` → final scalar output per example.

---

## 2. The forward method

```python
def forward(self, x):        # x: (B, T, F)
    _, hN = self.gru(x)      # hN: (num_layers, B, H);
    h = hN[-1]               # take last layer->(B, H)
    return self.head(h).squeeze(-1)  # (B,)
```

Step-by-step:

1. Input `x` shape: `(B, T, F)`

   * B = batch size
   * T = context_len
   * F = number of features (`len(FEATS)`)

2. `self.gru(x)` returns `(output, hN)`:

   * `output`: `(B, T, H)` (unused here)
   * `hN`: `(num_layers, B, H)` = final hidden per layer

3. `h = hN[-1]` picks the last layer’s final hidden state: shape `(B, H)`.

4. `self.head(h)` → shape `(B, 1)`.

5. `.squeeze(-1)` → removes the last dimension → `(B,)`, a 1D tensor of predictions.

So each batch gives you **one scalar prediction per sequence** (per window).

---

## 3. Building and moving the model to device

```python
def make_model():
    return GRURegressor(in_features=len(FEATS), hidden=64, num_layers=2, dropout=0.1)
```

* Uses the feature list `FEATS` to set correct `in_features`.

```python
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = make_model().to(device)
```

* Picks `"cuda"` if a GPU is available, else `"cpu"`.
* Moves all model parameters to that device.

---

## 4. Counting parameters

```python
sum(p.numel() for p in model.parameters()), device
```

* `model.parameters()` → iterator over all tensors in the model.
* `p.numel()` → number of elements in that tensor.
* `sum(...)` → total number of trainable parameters.

The expression returns a tuple: `(n_params, device)`.

---

## ⭐ Tiny check

In `forward`, we used `hN[-1]`.

### ❓ Why are we taking `hN[-1]` instead of `hN[0]`?

Pick the best answer:

A. `hN[-1]` gives the final time step of the sequence.\
B. `hN[-1]` gives the last layer’s hidden state, which is usually the most processed representation.\
C. `hN[-1]` saves memory.\
D. It doesn’t matter; any index would work.\

What do you think?
<details>
<summary> click for answer </summary>
 **B is the correct and important reason.**



#  Why we use `hN[-1]`

For an `nn.GRU` with `num_layers=2`,
the shape of the hidden state is:

```
hN: (num_layers, B, H)
```

That means:

* `hN[0]` = final hidden state of **layer 1**
* `hN[1]` = final hidden state of **layer 2** (the *top* layer)

And in general:

> **`hN[-1]` is the deepest representation the model has computed.
> It's the most expressive summary of the entire sequence.**

So this is the correct choice for downstream regression.

If you used `hN[0]`, you'd be using the shallowest layer's representation — less abstract, usually worse.



</details>


# 5) Training loop with AMP, early stopping, checkpointing

In [21]:
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
import math, time

def mae_t(y_true, y_pred): return torch.mean(torch.abs(y_true - y_pred))
def smape_t(y_true, y_pred, eps=1e-8):
    return torch.mean(2.0*torch.abs(y_true - y_pred)/(torch.abs(y_true)+torch.abs(y_pred)+eps))

def train_one_epoch(model, loader, optimizer, scaler, device, use_amp=True):  # scaler will be set as GradScaler
    model.train(); total=0.0; n=0
    for xb, yb, _ in loader:
        xb = xb.to(device, non_blocking=True).float()
        yb = yb.to(device, non_blocking=True).float()
        optimizer.zero_grad(set_to_none=True)
        if use_amp and device.type=="cuda":
            with autocast(dtype=torch.float16):
                pred = model(xb)
                loss = mae_t(yb, pred)  # train with MAE (robust)
            scaler.scale(loss).backward()  #scaler =GradScaler() see below in function fit()
            scaler.step(optimizer); scaler.update()
        else:
            pred = model(xb); loss = mae_t(yb, pred)
            loss.backward(); optimizer.step()
        bs = xb.size(0); total += loss.item()*bs; n += bs
    return total/max(n,1) #avg loss over all samples

@torch.no_grad() # turn off gradient tracking, see below for in-depth
def evaluate(model, loader, device):
    model.eval(); tot_mae=tot_smape=0.0; n=0
    for xb, yb, _ in loader:
        xb = xb.to(device, non_blocking=True).float()
        yb = yb.to(device, non_blocking=True).float()
        pred = model(xb)
        bs = xb.size(0)
        tot_mae   += mae_t(yb, pred).item()*bs
        tot_smape += smape_t(yb, pred).item()*bs
        n += bs
    return {"mae": tot_mae/max(n,1), "smape": tot_smape/max(n,1)}

def fit(model, train_loader, val_loader, epochs=12, lr=1e-3, wd=1e-5, patience=3, use_amp=True):
    opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scaler = GradScaler(enabled=use_amp and (device.type=="cuda"))
    best = math.inf; best_metrics=None; best_epoch=-1
    ckpt_path = Path("models/gru_split1.pt")
    history=[]
    for epoch in range(1, epochs+1):
        t0=time.time()
        tr_loss = train_one_epoch(model, train_loader, opt, scaler, device, use_amp)
        val = evaluate(model, val_loader, device)
        dt=time.time()-t0 # record training time for this epoch
        history.append({"epoch":epoch,"train_mae":tr_loss,"val_mae":val["mae"],"val_smape":val["smape"],"seconds":dt})
        print(f"Epoch {epoch:02d}  train_mae={tr_loss:.5f}  val_mae={val['mae']:.5f}  val_sMAPE={val['smape']:.5f}  ({dt:.1f}s)")
        # early stopping on val mae
        if val["mae"] < best - 1e-6:
            best = val["mae"]; best_metrics=val; best_epoch=epoch
            torch.save({
                "model_state": model.state_dict(),
                "optimizer_state": opt.state_dict(),
                "epoch": epoch,
                "val": val,
                "config": {"lr":lr,"wd":wd,"epochs":epochs,"context_len":train_loader.dataset.T,"feats":FEATS}
            }, ckpt_path)
        elif epoch - best_epoch >= patience:
            print(f"Early stopping at epoch {epoch} (best {best:.5f} @ {best_epoch})")
            break
    return history, best, best_epoch, ckpt_path

history, best, best_epoch, ckpt_path = fit(model, train_loader, val_loader,
                                           epochs=10, lr=1e-3, wd=1e-5, patience=3, use_amp=True)
print("Best val_mae:", best, "at epoch", best_epoch, "| saved:", ckpt_path.exists())

/tmp/ipython-input-253071626.py:42: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=use_amp and (device.type=="cuda"))


Epoch 01  train_mae=0.02655  val_mae=0.01088  val_sMAPE=1.15312  (0.5s)
Epoch 02  train_mae=0.01527  val_mae=0.00896  val_sMAPE=1.62450  (0.3s)
Epoch 03  train_mae=0.01282  val_mae=0.00781  val_sMAPE=1.46157  (0.3s)
Epoch 04  train_mae=0.01155  val_mae=0.00904  val_sMAPE=1.51492  (0.3s)
Epoch 05  train_mae=0.01127  val_mae=0.00890  val_sMAPE=1.52386  (0.3s)
Epoch 06  train_mae=0.01055  val_mae=0.01012  val_sMAPE=1.60047  (0.3s)
Early stopping at epoch 6 (best 0.00781 @ 3)
Best val_mae: 0.007809180611123641 at epoch 3 | saved: True


# ⭐ Taining function

---

## Big picture

```python
def train_one_epoch(model, loader, optimizer, scaler, device, use_amp=True):
    model.train(); total=0.0; n=0
    ...
    return total/max(n,1)
```

* Runs **one full pass** over `loader` (all training batches).
* Uses **MAE loss**.
* Optionally uses **AMP** (automatic mixed precision) when on CUDA.
* Returns the **average loss over all samples**.

---

## 1. Start of epoch

```python
model.train(); total=0.0; n=0
```

* `model.train()` → tells PyTorch we are in **training mode**
  (enables dropout, uses batchnorm in training mode, etc.)
* `total` will accumulate **sum of losses × batch_size**
* `n` will accumulate **number of samples**

So `total / n` at the end = mean MAE.

---

## 2. Loop over batches

```python
for xb, yb, _ in loader:
```

* `xb`: input features, shape `(B, T, F)`
* `yb`: targets, shape `(B,)`
* `_`: tickers (string labels), not used here

---

## 3. Move batch to device

```python
xb = xb.to(device, non_blocking=True).float()
yb = yb.to(device, non_blocking=True).float()
```

* Copies tensors to GPU (or CPU) — this is an **H2D copy** if on CUDA.
* `.float()` ensures `float32` (important before AMP casting).
* `non_blocking=True` can make H2D copies overlap with compute when using pinned memory. (see below for in-depth)

---

## 4. Zero the gradients

```python
optimizer.zero_grad(set_to_none=True)
```

* Clears old gradients before the backward pass.
* `set_to_none=True` is a small performance optimization vs setting grads to 0.

---

## 5. AMP vs non-AMP branch

### With AMP (GPU only)

```python
if use_amp and device.type=="cuda":
    with autocast(dtype=torch.float16):
        pred = model(xb)
        loss = mae_t(yb, pred)  # train with MAE (robust)
    scaler.scale(loss).backward() # scaler will be set as GradScaler()
    scaler.step(optimizer); scaler.update()
```

* `autocast(dtype=torch.float16)`:

  * Runs many ops in **fp16** internally where safe
  * Some critical ops stay in fp32 automatically
* `loss = mae_t(yb, pred)` → MAE loss on GPU.
* `scaler.scale(loss).backward()`:

  * multiplies loss by a scaling factor to avoid underflow in fp16
  * then backprop on scaled loss
* `scaler.step(optimizer)`:

  * steps optimizer with unscaled gradients (internally)
* `scaler.update()`:

  * adjusts scaling factor for next iteration if needed

This is the standard **GradScaler** pattern for mixed precision training.

---

### Without AMP (CPU or disabled)

```python
else:
    pred = model(xb); loss = mae_t(yb, pred)
    loss.backward(); optimizer.step()
```

Plain 32-bit training:

* Forward → loss → backward → optimizer step.

---

## 6. Accumulate epoch loss

```python
bs = xb.size(0)
total += loss.item() * bs
n += bs
```

* `bs` = batch size for this batch (may be smaller on last batch).
* `loss.item()` → scalar Python float (average loss over this batch).
* Multiply by `bs` → total loss contribution from all samples in batch.
* `total` = sum of loss over **all samples**.
* `n` = total number of samples seen.

---

## 7. Return mean loss

```python
return total/max(n,1)
```

* If `n > 0`, returns `total / n` = **mean MAE over epoch**.
* `max(n,1)` avoids division by 0 if loader is empty.

---

## One-sentence summary

> `train_one_epoch` iterates over training batches, optionally uses AMP on GPU, does forward + backward + optimizer step, and returns the mean MAE over all samples.

---



# ⭐ Quick check
In the AMP branch, why don’t we call `loss.backward()` directly, but instead:

```python
scaler.scale(loss).backward()
```

Pick the best reason:

A. To make gradients bigger so the optimizer converges faster.\
B. To avoid gradient underflow in fp16 by scaling the loss before backward.\
C. To make training deterministic.\
D. To change the learning rate each step.\

What do you think?
<details>
<summary> Click for answer </summary>
 **B is correct.**



---

#  Why we use `scaler.scale(loss).backward()` in AMP

In mixed-precision (fp16) training:

* fp16 has a **much smaller dynamic range**
* gradients can easily become **very tiny**
* tiny gradients may **underflow to zero** (become exactly 0), especially in deep networks

This destroys learning.

So PyTorch’s `GradScaler` solves it by:

1. **Scaling the loss upward**, e.g. multiplying by a large number like 1024.
2. Calling `.backward()` on the *scaled* loss.
3. Unscaling the gradients before the optimizer step.
4. Detecting overflows and adjusting scaling automatically.

Hence:

```python
scaler.scale(loss).backward()
```

prevents gradients from disappearing when using fp16.

---

# ⭐ Quick visual intuition

Without scaling:

```
true gradient = 0.00000018   # fits fine in FP32
fp16 → becomes 0.0           # underflow
```

With scaling:

```
scaled loss = loss * 1000
scaled gradient = 0.00018
fp16 → still representable
unscale later → correct gradient recovered
```



</details>


# ⭐ 1. What `non_blocking=True` actually means

When you do:

```python
xb = xb.to(device, non_blocking=True)
```

you are telling PyTorch:

> “If it is safe, please copy this tensor **asynchronously** from CPU → GPU.”

Meaning:

* The CPU **does not wait** for the copy to finish.
* The GPU can start the next operation sooner.
* Copies can overlap with GPU computation.

This can speed things up — *but only when certain conditions are met*.

---

# ⭐ 2. Why it matters

A normal H2D copy (`non_blocking=False`):

* blocks the CPU
* must finish before forward pass begins
* wastes time

With `non_blocking=True`:

* CPU starts the copy
* **immediately continues** to execute Python code
* GPU *may* overlap compute and data copy

It’s like saying:

> “Start the transfer and don’t freeze everything while it finishes.”

---

# ⭐ 3. When it actually works (important!)

`non_blocking=True` **only provides real overlap** if BOTH:

###  Condition 1: DataLoader uses **pin_memory=True**

Pinned memory = page-locked memory that CUDA can DMA (direct memory access, see below for in-depth)  directly:

```python
DataLoader(..., pin_memory=True)
```

Without pinned memory, CUDA cannot safely copy asynchronously.

###  Condition 2: You're copying from **CPU → GPU**

Async copy only applies to H2D transfers.
GPU→GPU transfers are already async.

###  Condition 3: The code after the `.to(device)` does not force synchronization

Some operations force a sync (e.g. printing a CUDA tensor), which removes the benefit.

---

# ⭐ 4. What “asynchronous” really means (simple mental model)

## Blocking copy

```
[CPU copies data] ----> wait ----> continue
```

## Non-blocking copy

```
[CPU starts copy] -> continue running Python
        \---- [GPU copy happens in background]
```

You get more overlap, like pipelining:

* worker loads next batch
* GPU computes on current batch
  -.cuda streams copy next batch while GPU is training

This speeds up training with medium/large models.

---

# ⭐ 5. Why deep learning frameworks use it

Because the faster your GPU is, the more dangerous the bottleneck becomes:

> The GPU often finishes compute **before** the next batch is ready.

Non-blocking transfer + pinned memory + multiple workers
helps eliminate that bottleneck.

---

# ⭐ 6. Very short explanation


> **`non_blocking=True` allows H2D copies to run asynchronously, overlapping with GPU compute, but only if the input tensors come from pinned CPU memory.**

---

# ⭐ Quick check

Which of the following is **required** for `non_blocking=True` to actually improve training speed?

A. GPU runtime must be slow \
B. Pinned memory must be enabled in DataLoader\
C. Model must be FP16\
D. Batch size must be at least 1024\

What do you think?

<details>
<summary> Click for answer </summary>
B
</details>


# ⭐ What is DMA?

**DMA = Direct Memory Access.**

It is a hardware feature that allows data to be moved:

* **directly** between system RAM ↔ GPU memory
* **without involving the CPU** in the actual transfer

This makes data movement:

* **faster**
* **more efficient**
* **non-blocking** (CPU stays free and can continue doing other work)

You can think of DMA like a “dedicated data mover” that copies memory independently.

---

# ⭐ Why do GPUs need DMA?

When you do:

```python
xb = xb.to("cuda", non_blocking=True)
```

this triggers a **host → device memory copy**.

Two scenarios:

---

## ❌ Without DMA:

* CPU must **read and push** each chunk of memory manually.
* CPU is **blocked** until the copy finishes.
* Slow and wasteful, especially when copying lots of batches.

---

##  With DMA:

The CPU says:

> “Hey DMA engine, copy this memory block from RAM into GPU memory —
> let me know when you’re done.”

And then the CPU **immediately continues** doing other work
(e.g., preparing the next batch, running Python code).

Meanwhile:

* DMA hardware moves the data **in the background**
* GPU can start computation as soon as the data arrives
* CPU and GPU are both used efficiently

---

# ⭐ How PyTorch uses DMA

DMA is used **automatically** when:

### ✔ 1. `pin_memory=True` in the DataLoader

Pinned (page-locked) memory is required for DMA.

###  2. Input tensors come from that pinned memory

DataLoader returns tensors allocated in pinned RAM.

###  3. You call `.to(device, non_blocking=True)`

This allows CUDA to perform a **DMA-based async copy**.

---

# ⭐ Why pinned memory is required for DMA?

Because:

* DMA can only read from **page-locked memory**
* Regular pageable RAM may be moved/swapped by OS
* GPU cannot DMA safely from pageable memory

So:

> **Pinned memory + non_blocking = true asynchronous DMA copy.**

Without pinned memory:

> `.to(device, non_blocking=True)` silently falls back to a blocking copy.

---

# ⭐ Visual intuition

### Without DMA

(Blocking, CPU copies bytes manually)

```
CPU: [copying 10 MB] ---- waiting ---- waiting ---- continue
GPU: idle
```

### With DMA

(Asynchronous copy)

```
CPU: Start DMA and continue → next batch, next Python steps
GPU: computing on previous batch
DMA engine: transferring new data in background
```

Everything overlaps — much faster.

---

# ⭐ One-sentence definition

> **DMA is hardware that copies memory directly between RAM and GPU without using the CPU, enabling fast asynchronous data transfer when used with pinned memory.**

---

# ⭐ Quick check (your turn)

Why does PyTorch need *pinned memory* for async H2D copies?

A. Because pinned memory never changes address, so DMA can safely read it\
B. Because pinned memory is faster to allocate\
C. Because pinned memory stores tensors on the GPU\
D. Because pinning memory makes tensors fp16

Which answer feels correct?

<details>
<summary> Click for answer </summary>
A
</details>



# ⭐ 1. `@torch.no_grad()`

This decorator tells PyTorch:

> "During this entire function, **do not track gradients**." (do not build computation graph)

Why?

* Evaluation/inference doesn’t need gradients
* Saves memory
* Speeds up computation
* Ensures no accidental `.backward()` occurs

So this whole function is **pure inference** — very efficient.

---

# ⭐ 2. `model.eval()`

This switches the model to **evaluation mode**:

* Disables dropout
* Turns batchnorm into "use running stats" mode
* Ensures deterministic, consistent behavior

Training mode: `model.train()`
Eval mode: `model.eval()`

These two change how some layers behave.


# ⭐ 4. Loop over batches

```python
for xb, yb, _ in loader:
```

Each batch:

* `xb`: inputs `(B, T, F)`
* `yb`: targets `(B,)`
* `_`: ticker names (ignored)

---

# ⭐ 5. Move to device

```python
xb = xb.to(device, non_blocking=True).float()
yb = yb.to(device, non_blocking=True).float()
```

Same idea as in training:

* H2D copy (async if pinned memory)
* Convert to float32
* No gradients tracked

---


# ⭐ 7. Compute metrics per batch

```python
bs = xb.size(0)
tot_mae   += mae_t(yb, pred).item() * bs
tot_smape += smape_t(yb, pred).item() * bs
n += bs
```

Important points:

* `mae_t(yb,pred).item()` → average MAE over the batch
* multiply by batch size → sum of MAE over samples
* accumulate over all validation batches
* keep track of total number of samples (`n`)

At the end, we divide:

```
tot_mae / n
```

to get the **true** dataset-wide MAE.

This is correct because not all batches may be the same size (especially last batch).


---



# ⭐ Quick comprehension check (your turn)

### ❓ Why do we wrap the entire function with `@torch.no_grad()`

instead of just using it inside the loop?

Pick the best reason:

A. It makes the code look nicer.\
B. It ensures *every* tensor operation inside this function is gradient-free, simplifying logic and ensuring correctness.\
C. It changes the learning rate.\
D. It forces the model to use FP16.

What do you think?

<details>
<summary> Click for answer </summary>
B
</details>



# ⭐ Fit funciton
```python
def fit(model, train_loader, val_loader, epochs=12, lr=1e-3, wd=1e-5, patience=3, use_amp=True):
    opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scaler = GradScaler(enabled=use_amp and (device.type=="cuda"))
    best = math.inf; best_metrics=None; best_epoch=-1
    ckpt_path = Path("models/gru_split1.pt")
    history=[]
    for epoch in range(1, epochs+1):
        t0=time.time()
        tr_loss = train_one_epoch(model, train_loader, opt, scaler, device, use_amp)
        val = evaluate(model, val_loader, device)
        dt=time.time()-t0
        history.append({"epoch":epoch,"train_mae":tr_loss,"val_mae":val["mae"],"val_smape":val["smape"],"seconds":dt})
        print(f"Epoch {epoch:02d}  train_mae={tr_loss:.5f}  val_mae={val['mae']:.5f}  val_sMAPE={val['smape']:.5f}  ({dt:.1f}s)")
        # early stopping on val mae
        if val["mae"] < best - 1e-6:
            best = val["mae"]; best_metrics=val; best_epoch=epoch
            torch.save({
                "model_state": model.state_dict(),
                "optimizer_state": opt.state_dict(),
                "epoch": epoch,
                "val": val,
                "config": {"lr":lr,"wd":wd,"epochs":epochs,"context_len":train_loader.dataset.T,"feats":FEATS}
            }, ckpt_path)
        elif epoch - best_epoch >= patience:
            print(f"Early stopping at epoch {epoch} (best {best:.5f} @ {best_epoch})")
            break
    return history, best, best_epoch, ckpt_path
```

---

## 1. Optimizer + GradScaler

```python
opt = AdamW(model.parameters(), lr=lr, weight_decay=wd)
```

* Uses **AdamW** (Adam with decoupled weight decay).
* `weight_decay=wd` is your L2-style regularization.

```python
scaler = GradScaler(enabled=use_amp and (device.type=="cuda"))
```

* `GradScaler` handles **loss scaling** for AMP.
* It’s only enabled if `use_amp=True` **and** you’re on CUDA.
* On CPU, it silently does nothing (no AMP).

---

## 2. Tracking the best model

```python
best = math.inf
best_metrics = None
best_epoch = -1
ckpt_path = Path("models/gru_split1.pt")
history = []
```

* `best` = best validation MAE seen so far (initialized to ∞).
* `best_metrics` = dict with MAE/SMAPE at best epoch.
* `best_epoch` = epoch number of best model.
* `history` = list of per-epoch logs (for plotting later).

---

## 3. Epoch loop

```python
for epoch in range(1, epochs+1):
    t0 = time.time()
    tr_loss = train_one_epoch(model, train_loader, opt, scaler, device, use_amp)
    val = evaluate(model, val_loader, device)
    dt = time.time() - t0
```

Per epoch:

* Call `train_one_epoch` → returns **train MAE**.
* Call `evaluate` → returns dict with `{"mae": ..., "smape": ...}`.
* `dt` = wall-clock time for this epoch.

---

## 4. Save to history + print

```python
history.append({
    "epoch": epoch,
    "train_mae": tr_loss,
    "val_mae": val["mae"],
    "val_smape": val["smape"],
    "seconds": dt
})
print(f"Epoch {epoch:02d}  train_mae={tr_loss:.5f}  val_mae={val['mae']:.5f}  val_sMAPE={val['smape']:.5f}  ({dt:.1f}s)")
```

* `history` keeps a full record you can later turn into a DataFrame or plot.
* The print gives a compact training log.

---

## 5. Early stopping + checkpoint saving

```python
# early stopping on val mae
if val["mae"] < best - 1e-6:
    best = val["mae"]; best_metrics = val; best_epoch = epoch
    torch.save({
        "model_state": model.state_dict(),
        "optimizer_state": opt.state_dict(),
        "epoch": epoch,
        "val": val,
        "config": {"lr":lr, "wd":wd, "epochs":epochs,
                   "context_len":train_loader.dataset.T, "feats":FEATS}
    }, ckpt_path)
elif epoch - best_epoch >= patience:
    print(f"Early stopping at epoch {epoch} (best {best:.5f} @ {best_epoch})")
    break
```

Two main ideas:

###  5.1. Model improvement

```python
if val["mae"] < best - 1e-6:
```

* If validation MAE **improves** by at least `1e-6`:

  * Update `best`
  * Record `best_metrics`, `best_epoch`
  * Save a checkpoint with:

    * `model_state` (weights)
    * `optimizer_state` (momentum, etc.)
    * `epoch` and `val` metrics
    * `config` (hyperparameters, feature config)

This way, even if later epochs get worse, you have the **best model** saved.

###  5.2. Patience-based early stopping

```python
elif epoch - best_epoch >= patience:
    print(f"Early stopping at epoch {epoch} (best {best:.5f} @ {best_epoch})")
    break
```

* If **no improvement** for `patience` epochs in a row:

  * Stop training early
  * Avoid overfitting and wasted compute

Example: `patience=3`

* If best epoch is 4 and you’re now at epoch 7:

  * 7 - 4 = 3 → stop at 7

---

## 6. Return values

```python
return history, best, best_epoch, ckpt_path
```

You get:

* `history` → full training curve
* `best` → best validation MAE
* `best_epoch` → where that happened
* `ckpt_path` → where the best model was saved

---

## Tiny check for you

The early stopping condition is:

```python
elif epoch - best_epoch >= patience:
```

### ❓ Why do we use `epoch - best_epoch` here?

A. To stop after a fixed number of total epochs.\
B. To stop after "patience" epochs **without any improvement** in validation MAE.\
C. To slow down the optimizer.\
D. To change the learning rate schedule.

Which one feels correct?
<details>
<summary> Click for answer </summary>
B
</details>


# 6) Evaluate best checkpoint & write a small report

In [22]:
# Reload best checkpoint and compute final validation metrics + save CSV
ckpt = torch.load("models/gru_split1.pt", map_location="cpu")
model.load_state_dict(ckpt["model_state"])
model.to(device)
final = evaluate(model, val_loader, device)
import pandas as pd
rep = pd.DataFrame([{
    "split": 1,
    "context_len": train_loader.dataset.T,
    "feats": ",".join(FEATS),
    "val_mae": final["mae"],
    "val_smape": final["smape"],
    "best_epoch": ckpt.get("epoch", None),
    "params_M": round(sum(p.numel() for p in model.parameters())/1e6, 3)
}])
rep.to_csv("reports/gru_split1_metrics.csv", index=False)
rep

,split,context_len,feats,val_mae,val_smape,best_epoch,params_M
0,1,16,"log_return,lag1,lag2,lag3,zscore_20,roll_std_20",0.007809,1.461566,3,0.043


# ⭐ 1. Load checkpoint from disk

```python
ckpt = torch.load("models/gru_split1.pt", map_location="cpu")
```

###  What it does:

* Loads the dictionary you saved earlier inside `fit()`.
* `map_location="cpu"` forces all tensors to CPU (safe and portable).

  * You later move the model to GPU if needed.
* The checkpoint contains:

  * `"model_state"` → a state_dict of weights
  * `"optimizer_state"` → optimizer buffers (you may or may not use this)
  * `"epoch"` → best epoch
  * `"val"` → metrics at that epoch
  * `"config"` → hyperparameters, feature config, etc.

---

# ⭐ 2. Load the model weights

```python
model.load_state_dict(ckpt["model_state"])
model.to(device)
```

###  What it does:

1. Loads the saved weights into your model architecture.
2. Moves the model to the appropriate device (`cuda` or `cpu`).

⚠️ Important:
The **model architecture must match** the architecture used during training
(same hidden size, layers, features, etc.), or this will error.

---

# ⭐ 3. Evaluate the model

```python
final = evaluate(model, val_loader, device)
```

This computes:

* final MAE
* final SMAPE
* using the **whole validation set**
* without gradients (thanks to `@torch.no_grad()` inside `evaluate`)

This is your unbiased, final validation score.

---

# ⭐ 4. Create a Pandas report row

```python
import pandas as pd
rep = pd.DataFrame([{
    "split": 1,
    "context_len": train_loader.dataset.T,
    "feats": ",".join(FEATS),
    "val_mae": final["mae"],
    "val_smape": final["smape"],
    "best_epoch": ckpt.get("epoch", None),
    "params_M": round(sum(p.numel() for p in model.parameters())/1e6, 3)
}])
```

### ✔ Column-by-column explanation:

* `"split": 1`
  If you're running multiple data splits, this tracks which one.

* `"context_len": train_loader.dataset.T`
  The window size used (e.g., 32 or 64).

* `"feats": ",".join(FEATS)`
  Stores the feature list as a single string.

* `"val_mae": final["mae"]`
  The final MAE (after restoring best checkpoint).

* `"val_smape": final["smape"]`
  SMAPE metric.

* `"best_epoch": ckpt.get("epoch", None)`
  Epoch number where early stopping found the best model.

* `"params_M": ...`
  Number of parameters in megabytes:

  ```python
  sum(p.numel() for p in model.parameters()) / 1e6
  ```

This is a clean, reproducible experiment summary row.

---

# ⭐ 5. Save to CSV

```python
rep.to_csv("reports/gru_split1_metrics.csv", index=False)
rep
```

* Saves the metrics for this run into a report file.
* Returning `rep` prints the DataFrame nicely in the notebook.

This is nice because:

* All results across models, features, window lengths, etc., become easy to compare.
* You can aggregate multiple split results later into one summary.

---

# ⭐ One-Sentence Summary

> You load the best checkpoint, restore the model, evaluate it, record its final performance, and save a clean experiment summary.

---

# ⭐ Quick comprehension check (your turn)

Why do we use `map_location="cpu"` when loading the checkpoint,
even if we later want to evaluate on the GPU?

Pick the best answer:

A. CPU loading is faster.\
B. The checkpoint always stores CPU weights.\
C. It makes loading safe even if the checkpoint was saved on a different machine or GPU configuration.\
D. It automatically improves MAE.

Which one feels correct?

<details>
<summary> Click for answer </summary>
C
</details>


> **Time check:** With \~5–8 tickers, `T=64`, and 10 epochs, this should finish in a couple of minutes on Colab CPU; faster on GPU.

------------------------------------------------------------------------

## Wrap‑up

-   **WindowedDataset** emits causal windows `(≤ t)` and targets `r_1d[t]` (i.e., `t+1` return).
-   Use **train‑fit scaler** and **reuse it** on validation to avoid leakage.
-   Keep the training loop **simple**: MAE training objective, AMP on CUDA, **early stopping** on validation MAE, and save a **checkpoint**.
-   Produce a **CSV** with validation metrics to track progress and compare future models.

------------------------------------------------------------------------

## Homework (due before Session 20)

**Goal:** Train a stronger **sequence baseline** (choose **LSTM** or **TCN**) on a subset (5–10 tickers). Log metrics and push your checkpoint + report.

### Part A — Script `scripts/train_seq.py` (LSTM or TCN)


In [ ]:
#!/usr/bin/env python
from __future__ import annotations
import argparse, json, math
from pathlib import Path
import numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler

# --- (reuse minimal dataset/scaler from class; compact copy here) ---
class FeatureScaler:
    def __init__(self): self.mean_=None; self.std_=None
    def fit(self, X): self.mean_=X.mean(0); self.std_=X.std(0)+1e-8; return self
    def transform(self, X): return (X-self.mean_)/self.std_
    def state_dict(self): return {"mean": self.mean_.tolist(), "std": self.std_.tolist()}
    def load_state_dict(self, d): import numpy as np; self.mean_=np.array(d["mean"]); self.std_=np.array(d["std"])
class WindowedDataset(Dataset):
    def __init__(self, df, feats, T=64, scaler=None):
        self.feats=feats; self.T=T; self.idx=[]; self.g={}
        for tkr,g in df.groupby("ticker"):
            g=g.sort_values("date").reset_index(drop=True)
            X=g[feats].to_numpy("float32"); y=g["r_1d"].to_numpy("float32")
            for end in range(T-1,len(g)):
                if np.isfinite(y[end]): self.idx.append((tkr,end))
            self.g[tkr]={"X":X,"y":y}
        self.scaler = scaler or FeatureScaler().fit(np.concatenate([self.g[t]["X"] for t in self.g],0))
    def __len__(self): return len(self.idx)
    def __getitem__(self,i):
        tkr,end=self.idx[i]; g=self.g[tkr]
        X=g["X"][end-self.T+1:end+1]; X=self.scaler.transform(X)
        y=g["y"][end]
        return torch.from_numpy(X), torch.tensor(y)

def make_splits(dates, train_min=252, val_size=63, step=63, embargo=5):
    u=np.array(sorted(pd.to_datetime(pd.Series(dates).unique()))); i=train_min-1; out=[]
    while True:
        if i>=len(u): break
        a,b=u[0],u[i]; vs=i+embargo+1; ve=vs+val_size-1
        if ve>=len(u): break
        out.append((a,b,u[vs],u[ve])); i+=step
    return out

# --- Models ---
class LSTMReg(nn.Module):
    def __init__(self, in_f, hidden=64, layers=2, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(in_f, hidden, num_layers=layers, batch_first=True, dropout=dropout if layers>1 else 0.)
        self.head = nn.Sequential(nn.Linear(hidden, hidden), nn.ReLU(), nn.Dropout(dropout), nn.Linear(hidden,1))
    def forward(self, x):
        out,_ = self.lstm(x)
        h = out[:,-1,:]
        return self.head(h).squeeze(-1)

class TCNBlock(nn.Module):
    def __init__(self, in_c, out_c, k=3, d=1, dropout=0.1):
        super().__init__()
        pad = (k-1)*d
        self.net = nn.Sequential(
            nn.Conv1d(in_c, out_c, k, padding=pad, dilation=d),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Conv1d(out_c, out_c, k, padding=pad, dilation=d),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.down = nn.Conv1d(in_c, out_c, 1) if in_c!=out_c else nn.Identity()
    def forward(self, x):        # x: (B, F, T)
        y = self.net(x)
        # Causal crop to ensure output aligns with last time step
        crop = y.shape[-1]-x.shape[-1]
        if crop>0: y = y[..., :-crop]
        return y + self.down(x)

class TCNReg(nn.Module):
    def __init__(self, in_f, ch=64, blocks=3, k=3, dropout=0.1):
        super().__init__()
        layers=[]; c=in_f
        for b in range(blocks):
            layers.append(TCNBlock(c, ch, k=k, d=2**b, dropout=dropout)); c=ch
        self.tcn = nn.Sequential(*layers)
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(1), nn.Flatten(), nn.Linear(ch,1))
    def forward(self, x):
        # x: (B,T,F) -> (B,F,T) for Conv1d
        x = x.transpose(1,2)
        y = self.tcn(x)              # (B, C, T)
        return self.head(y).squeeze(-1)

def mae_t(y,yhat): return torch.mean(torch.abs(y - yhat))
def smape_t(y,yhat,eps=1e-8): return torch.mean(2*torch.abs(y-yhat)/(torch.abs(y)+torch.abs(yhat)+eps))

def main():
    ap=argparse.ArgumentParser()
    ap.add_argument("--features", default="data/processed/features_v1.parquet")
    ap.add_argument("--context", type=int, default=64)
    ap.add_argument("--model", choices=["lstm","tcn"], default="lstm")
    ap.add_argument("--epochs", type=int, default=12)
    ap.add_argument("--batch", type=int, default=256)
    ap.add_argument("--lr", type=float, default=1e-3)
    ap.add_argument("--patience", type=int, default=3)
    ap.add_argument("--tickers", type=int, default=8)
    args=ap.parse_args()

    df = pd.read_parquet("data/processed/features_v1_static.parquet") if Path("data/processed/features_v1_static.parquet").exists() else pd.read_parquet(args.features)
    df = df.sort_values(["ticker","date"]).reset_index(drop=True)
    cand = ["log_return","lag1","lag2","lag3","zscore_20","roll_std_20"]
    feats = [c for c in cand if c in df.columns]
    assert "r_1d" in df.columns
    # subset tickers
    keep = df["ticker"].astype(str).unique().tolist()[:args.tickers]
    df = df[df["ticker"].astype(str).isin(keep)].copy()

    splits = make_splits(df["date"])
    a,b,c,d = splits[0]
    tr = df[(df["date"]>=a)&(df["date"]<=b)]
    va = df[(df["date"]>=c)&(df["date"]<=d)]

    train_ds = WindowedDataset(tr, feats, T=args.context, scaler=None)
    val_ds   = WindowedDataset(va, feats, T=args.context, scaler=train_ds.scaler)

    pin = torch.cuda.is_available()
    g = torch.Generator(); g.manual_seed(42)
    def _seed_worker(_): import numpy as np, random, torch; ws=torch.initial_seed()%2**32; np.random.seed(ws); random.seed(ws)
    train_ld = DataLoader(train_ds, batch_size=args.batch, shuffle=True, drop_last=True,
                          num_workers=2, pin_memory=pin, worker_init_fn=_seed_worker, generator=g)
    val_ld   = DataLoader(val_ds, batch_size=args.batch, shuffle=False, drop_last=False,
                          num_workers=2, pin_memory=pin, worker_init_fn=_seed_worker)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = (LSTMReg(len(feats)) if args.model=="lstm" else TCNReg(len(feats))).to(device)
    opt = torch.optim.AdamW(net.parameters(), lr=args.lr, weight_decay=1e-5)
    scaler = GradScaler(enabled=(device.type=="cuda"))
    best = 1e9; best_epoch=0
    ckpt = Path(f"models/{args.model}_split1.pt")

    for epoch in range(1, args.epochs+1):
        net.train(); tmae=0; n=0
        for xb,yb in train_ld:
            xb=xb.to(device).float(); yb=yb.to(device).float()
            opt.zero_grad(set_to_none=True)
            with autocast(enabled=(device.type=="cuda"), dtype=torch.float16):
                yhat = net(xb)
                loss = mae_t(yb, yhat)
            scaler.scale(loss).backward(); scaler.step(opt); scaler.update()
            bs=xb.size(0); tmae += loss.item()*bs; n+=bs
        tr_mae=tmae/n
        # val
        net.eval(); vmae=vsm=0; n=0
        with torch.no_grad():
            for xb,yb in val_ld:
                xb=xb.to(device).float(); yb=yb.to(device).float()
                yhat = net(xb)
                bs=xb.size(0); vmae += mae_t(yb,yhat).item()*bs; vsm += smape_t(yb,yhat).item()*bs; n+=bs
        vmae/=n; vsm/=n
        print(f"Epoch {epoch:02d}  tr_mae={tr_mae:.5f}  val_mae={vmae:.5f}  val_sMAPE={vsm:.5f}")
        if vmae < best-1e-6:
            best=vmae; best_epoch=epoch
            torch.save({"model": net.state_dict(), "epoch": epoch, "feats": feats, "context": args.context}, ckpt)
        elif epoch - best_epoch >= args.patience:
            print("Early stopping.")
            break

    Path("reports").mkdir(exist_ok=True)
    pd.DataFrame([{"model":args.model,"context":args.context,"val_mae":best,"best_epoch":best_epoch,"feats":",".join(feats)}]).to_csv(
        f"reports/{args.model}_split1_metrics.csv", index=False)

if __name__ == "__main__":
    main()

In [ ]:
%%bash
chmod +x scripts/train_seq.py
python scripts/train_seq.py --model lstm --context 64 --tickers 8 --epochs 12

### Part B — Add a quick **Makefile** target and a tiny test

**Append to `Makefile`:**

``` make
.PHONY: train-lstm
train-lstm: ## Train LSTM baseline on split 1 (subset of tickers)
\tpython scripts/train_seq.py --model lstm --context 64 --tickers 8 --epochs 12
```




**Basic shape test for dataset windows:**

In [ ]:
# tests/test_windowed_dataset.py
import pandas as pd, numpy as np, os
def test_window_shapes():
    import scripts.train_seq as T
    df = pd.read_parquet("data/processed/features_v1.parquet").sort_values(["ticker","date"]).reset_index(drop=True)
    feats = [c for c in ["log_return","lag1","lag2","lag3"] if c in df.columns]
    splits = T.make_splits(df["date"])
    a,b,c,d = splits[0]
    ds = T.WindowedDataset(df[(df["date"]>=a)&(df["date"]<=b)], feats, T=32, scaler=None)
    X,y = ds[0]
    assert X.shape == (32, len(feats))
    assert np.isfinite(y.item())

In [ ]:
%%bash
pytest -q -k windowed_dataset

### Part C — Report

Add to your Quarto report (e.g., `reports/eda.qmd`):

````markdown \## PyTorch Baselines

```{python}
import pandas as pd
print(pd.read_csv("reports/gru_split1_metrics.csv"))
try:
    print(pd.read_csv("reports/lstm_split1_metrics.csv"))
except Exception as e:
    print("lstm metrics not found yet")

```
````